<a href="https://colab.research.google.com/github/SrijaG29/offc_dsa_prac/blob/main/Welcome_To_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Isomorphic Strings

In [ ]:
import java.util.ArrayList;
import java.util.HashMap;
import java.util.List;
import java.util.Map;

public class isomorphicString {
    public static void main(String[] args) {
        String s = "paper";
        String t = "title";

        if (s.length() != t.length()) {
            System.out.println("No");
        }

        Map<Character, Character> mapSToT = new HashMap<>();
        Map<Character, Character> mapTToS = new HashMap<>();

        for (int i = 0; i < s.length(); i++) {
            char charS = s.charAt(i);
            char charT = t.charAt(i);
            if (mapSToT.containsKey(charS)) {
                if (mapSToT.get(charS) != charT) {
                    System.out.println("No");
                }
            } else {
                mapSToT.put(charS, charT);
            }
            if (mapTToS.containsKey(charT)) {
                if (mapTToS.get(charT) != charS) {
                    System.out.println("No");
                }
            } else {
                mapTToS.put(charT, charS);
            }
        }
        System.out.println("Yes");
    }
}


Html code

In [ ]:
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Expense Calculator</title>
    <link rel="stylesheet" href="style.css">
</head>
<body>
    <div class="navbar">
        <img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRqV8qU3m3W7cqHR9zkcdtry3bP-rQGybA-sw&s"
        alt="Logo" id="image">
        <h1 class="mainHeading">Expenses Calculator!</h1>
    </div>
    <div id="details">
        <div class="expensesForm">
            <h3 class="heading">Expenses Form</h3>
            <form class="formElements" id="formid" onsubmit="getdetails(event)">
               <div class="select">
                    <label for="expenseName">Expense Name: </label>
                    <input type="text" id="expenseName" name="expenseName" required>
               </div>
               <div class="select">
                    <label for="options">Select a category:</label>
                    <select id="options" >
                        <option value="Select a Category">Select a Category</option>
                        <option value="Dressing">Dressing</option>
                        <option value="Food">Food</option>
                        <option value="Travel">Travel</option>
                        <option value="Other">Other</option>
                    </select>
                </div>
                <div class="select" id="otherInputContainer"></div>
                <div class="select">
                    <label for="totalAmount">Total Amount: </label>
                    <input type="number" id="totalAmount" name="totalAmount" required>
                </div>
                <div class="select">
                    <button type="submit">Submit</button>
                </div>
            </form>
        </div>
        <div class="additionalDetails">
            <div class="firstpart">
                <div class="filtering">
                    <p>Filtering options:</p>
                    <select id="filteroptions" >
                        <option value="Select a Category">Select a Category</option>
                        <option value="Dressing">Dressing</option>
                        <option value="Food">Food</option>
                        <option value="Travel">Travel</option>
                    </select>
                </div>
                <div class="values">
                    <div class="expensedetails">
                        <table>
                            <tr>
                                <th><b>Expenses</b></th>
                                <th><b>Category</b></th>
                                <th><b>Total Amount</b></th>
                            </tr>
                        </table>

                    </div>
                </div>
            </div>
            <div class="secondpart">
                    <div id="dropdowns">
                        <label for="monthdropdowns">Select Month:</label>
                        <select id="monthDropdown">
                            <option value="MM">MM</option>
                            <option value="January">January</option>
                            <option value="February">February</option>
                            <option value="March">March</option>
                            <option value="April">April</option>
                            <option value="May">May</option>
                            <option value="June">June</option>
                            <option value="July">July</option>
                            <option value="August">August</option>
                            <option value="Sepetember">Sepetember</option>
                            <option value="October">October</option>
                            <option value="November">November</option>
                            <option value="December">December</option>
                        </select>
                        <label for="monthdropdowns">Select Year:</label>
                        <select id="yearDropdown"></select>
                    </div>
                    <div id="monthwisecontent">
                    </div>
            </div>
        </div>
    </div>
    <script src="actions.js"></script>
</body>
</html>





css code

In [ ]:
body{
    display: flex;
    flex-direction: column;
    background-color: rgb(205, 231, 234);
    margin: 0;
    width: 100%;
}

.navbar{
    /* height: 30%; */
    display: flex;
    border-bottom: 2px solid black;
    /* padding-top: 2.5%;
    padding-bottom: 2.5%; */
    justify-content: flex-start;
    align-items: center;
    background-color: rgb(6, 55, 70);
    color: azure;
}

#image{
    height: 50px;
    width: 100px;
    border-radius: 20%;
    padding: 0 2%;

}

.heading{
    margin: 0;
    padding-bottom: 1%;
}

.expensesForm{
    height: 30%;
    padding: 1%;
    border-bottom: 2px solid black;
}

.formElements{
    display: flex;
    justify-content: space-between;
}
.select{
    display: flex;
    /* justify-content: center; */
    /* justify-content: space-between; */
}


#otherinput{
    border-width: 1px;
    border-style: solid;
    border-color: light-dark(rgb(118, 118, 118)),
}

.additionalDetails{
    display: flex;
    width: 100%;
}

.firstpart{
    width: 50%;
    border-right: 2px solid black;
}

.filtering{
    display: flex;
    justify-content: center;
    padding-top: 5%;
    height: 25%;
    align-items: center;
}

.values{
    justify-items: center;
    height: 75%;
    padding-top: 5%;
    padding-bottom: 5%;
}

#filteroptions{
    width: 40%;
    height: 30px;
    margin-left: 10px;
}

.secondpart{
    width: 50%;
}

.expensedetails{
    padding:10px;
}
.expensedetails th{
    border: 1px solid black; padding: 8px; text-align: left;
}

.expensedetails td {
    border: 1px solid black;
    padding: 8px;
    text-align: left;
}

#dropdowns{
    display: flex;
    padding-top: 5%;
    padding-left: 2%;
    padding-right: 2%;
    align-items: center;
    height: 25%;
}

/*#monthDropdown{
    width: 50%;
}

#yeardropdown{
    display: flex;
    text-align: center;
    padding-top: 5%;
    width: 50%;
    justify-content: space-evenly;
    text-align: center;
}

#yearDropdown{
    width: 50%;
}

.textval{
    padding-right: 2%;
    width: 50%;
}*/

#monthwisecontent{
    height: 75%;
}


/*
<div id="monthdropdown">
    <p class="textval">Select Month:</p>
    <select id="monthDropdown">
        <option value="MM">MM</option>
        <option value="January">January</option>
        <option value="February">February</option>
        <option value="March">March</option>
        <option value="April">April</option>
        <option value="May">May</option>
        <option value="June">June</option>
        <option value="July">July</option>
        <option value="August">August</option>
        <option value="Sepetember">Sepetember</option>
        <option value="October">October</option>
        <option value="November">November</option>
        <option value="December">December</option>
    </select>
</div>
<div id="yeardropdown">
    <p class="textval">Select Year:</p>
    <select id="yearDropdown"></select>
</div>
*/


javascript

In [ ]:
// This is to take only aphabelts in expensename element
let expenseNameInput = document.getElementById("expenseName");
expenseNameInput.addEventListener("input", function(event) {
    // Remove any non-alphabet characters
    event.target.value = event.target.value.replace(/[^A-Za-z ]+/g, '');
});



let options = document.getElementById("options");
let newele;
let newlabel;
options.onchange = () => {
    const container = document.getElementById("otherInputContainer");
    // Clear the container before adding new elements
    container.innerHTML = '';
    if (options.value === "Other") {
        // Create and append a label
        newlabel = document.createElement("label");
        newlabel.textContent = "Other Category:";
        container.appendChild(newlabel);
        // Create and append a textarea
        let newele = document.createElement("input");
        newele.type = "text";
        newele.setAttribute("id","otherinput")
        newele.setAttribute("required",true);
        // This is to take only aphabelts in otherinput textarea element
        container.addEventListener("input", function(event) {
            // Remove any non-alphabet characters
        event.target.value = event.target.value.replace(/[^A-Za-z ]+/g, '');
        });
        container.appendChild(newele);
    }
};

let index = 0;
let details = {}
let table = document.querySelector(".expensedetails table");
function getdetails(event){
    event.preventDefault()
    let category
    if(document.getElementById("options").value === "Other"){
        category = document.getElementById("otherinput").value
        // adding new category into options for future selection in form
        let optionselement = document.getElementById("options")
        let categoryoption = document.createElement("option")
        categoryoption.textContent = category
        categoryoption.value = category
        let optionExists = Array.from(optionselement.options).some(option => option.value === category);
        if (!optionExists) {
            // Insert the new option at the second-to-last position
            optionselement.insertBefore(categoryoption,optionselement.lastElementChild)
        }else {
            // If the option already exists, just set the dropdown value to the existing option
            optionselement.value = category;
        }


        // adding new category into options for future selection in filtering
        let optionsfilteringelement = document.getElementById("filteroptions")
        let categoryfilteringoption = document.createElement("option")
        categoryfilteringoption.textContent = category
        categoryfilteringoption.value = category
        let filteringOptionExists = Array.from(optionsfilteringelement.options).some(option => option.value === category);
        if (!filteringOptionExists) {
            // Insert the new option at the second-to-last position
            optionsfilteringelement.insertBefore(categoryfilteringoption,optionsfilteringelement.lastElementChild)
        }else {
            // If the option already exists, just set the dropdown value to the existing option
            optionsfilteringelement.value = category;
        }
    }
    else{
        category = document.getElementById("options").value
    }
    // console.log(category)
    let d = {
        expenseName: document.getElementById("expenseName").value,
        category: category,
        totalAmount: document.getElementById("totalAmount").value
    };
    details[index] = d;
    index = index + 1;
    // console.log(details)
    expensedetails(d.expenseName,d.category,d.totalAmount) //
}

function expensedetails(expenses, category, totalAmount){ //
    // Create a new row directly inside the .expensedetails div (not inside <tbody>)
    let newRow = document.createElement("tr"); // Create a new row element
    let cell1 = document.createElement("td"); // Create the first cell
    let cell2 = document.createElement("td"); // Create the second cell
    let cell3 = document.createElement("td"); // Create the third cell

    cell1.innerHTML = expenses; // Add the expense name to the first cell
    cell2.innerHTML = category; // Add the category to the second cell
    cell3.innerHTML = totalAmount; // Add the total amount to the third cell

    // Append cells to the row
    newRow.appendChild(cell1);
    newRow.appendChild(cell2);
    newRow.appendChild(cell3);

    newRow.setAttribute("class","tablerows");
    // Append the new row directly to the table (the div in your case)
    table.appendChild(newRow);
}


let filtervalue = document.getElementById("filteroptions");
filtervalue.onchange = () => {
    let rows = document.querySelectorAll("tr.tablerows");
    for(let i = 0; i < rows.length; i++) {
        const row = rows[i];
        const categoryvalue = row.querySelector("td:nth-child(2)").textContent.trim();

        if (filtervalue.value === "Select a Category") {
            row.style.display = ""; // Reset display for all rows
        } else if (categoryvalue === filtervalue.value) {
            row.style.display = ""; // Show row if category matches
        } else {
            row.style.display = "none"; // Hide row if category doesn't match
        }
    }
};


function populateYearDropdown() {
    let currentYear = new Date().getFullYear();
    let startYear = 2000;
    let yearDropdown = document.getElementById('yearDropdown');

    for (let year = startYear; year <= currentYear; year++) {
        let option = document.createElement('option');
        option.value = year;
        option.textContent = year;
        yearDropdown.appendChild(option);
    }
}

document.addEventListener("DOMContentLoaded", populateYearDropdown);








